# **Project of Business Analytics**

## Importorting the necessary libarbraries

In [20]:
import pandas as pd
from geopy.geocoders import Nominatim
from time import sleep

## Collecting the data

### Water Consumption Data

In [2]:
# Collect the data using water_consumption data link

URL_DATA = 'https://storage.data.gov.my/water/water_consumption.parquet'

water_data = pd.read_parquet(URL_DATA)
if 'date' in water_data.columns: water_data['date'] = pd.to_datetime(water_data['date'])

print(water_data)

           state       sector       date  value
0       Malaysia     domestic 2003-01-01   4394
1       Malaysia     domestic 2004-01-01   4770
2       Malaysia     domestic 2005-01-01   4992
3       Malaysia     domestic 2006-01-01   5025
4       Malaysia     domestic 2007-01-01   5381
..           ...          ...        ...    ...
595  W.P. Labuan  nondomestic 2018-01-01     33
596  W.P. Labuan  nondomestic 2019-01-01     36
597  W.P. Labuan  nondomestic 2020-01-01     40
598  W.P. Labuan  nondomestic 2021-01-01     32
599  W.P. Labuan  nondomestic 2022-01-01     33

[600 rows x 4 columns]


### Crime Data

In [3]:
# Collect the data using crime_district data link

URL_DATA = 'https://storage.data.gov.my/publicsafety/crime_district.parquet'

crime_data = pd.read_parquet(URL_DATA)
if 'date' in crime_data.columns: crime_data['date'] = pd.to_datetime(crime_data['date'])

print(crime_data)

                   state     district  category                      type   
0               Malaysia          All   assault                       all  \
1               Malaysia          All   assault                       all   
2               Malaysia          All   assault                       all   
3               Malaysia          All   assault                       all   
4               Malaysia          All   assault                       all   
...                  ...          ...       ...                       ...   
19147  W.P. Kuala Lumpur  Wangsa Maju  property  theft_vehicle_motorcycle   
19148  W.P. Kuala Lumpur  Wangsa Maju  property  theft_vehicle_motorcycle   
19149  W.P. Kuala Lumpur  Wangsa Maju  property  theft_vehicle_motorcycle   
19150  W.P. Kuala Lumpur  Wangsa Maju  property  theft_vehicle_motorcycle   
19151  W.P. Kuala Lumpur  Wangsa Maju  property  theft_vehicle_motorcycle   

            date  crimes  
0     2016-01-01   22327  
1     2017-01-01   21

## Merge Water Consumption Data with Crime Data

In [4]:
# Extract year from date column
water_data["year"] = pd.to_datetime(water_data["date"]).dt.year
crime_data["year"] = pd.to_datetime(crime_data["date"]).dt.year

In [14]:
# Aggregate crime data at state level
crime_data_agg = crime_data.groupby(["state", "year", "category", "type"]).agg({"crimes": "sum"}).reset_index()

# Merge datasets on state and year, keeping relevant columns
merged_data = pd.merge(
    crime_data_agg,
    water_data[["state", "sector", "year", "value"]],
    on=["state", "year"],
    how="inner"
)

In [15]:
# Rename value column to water_consumption for clarity
merged_data.rename(columns={"value": "water_consumption"}, inplace=True)

In [16]:
# Save cleaned dataset
merged_data.to_csv("merged_water_crime.csv", index=False)

In [17]:
# Display sample output
merged_data

,state,year,category,type,crimes,sector,water_consumption
0,Johor,2016,assault,all,5640,domestic,773
1,Johor,2016,assault,all,5640,nondomestic,513
2,Johor,2016,assault,causing_injury,1416,domestic,773
3,Johor,2016,assault,causing_injury,1416,nondomestic,513
4,Johor,2016,assault,murder,140,domestic,773
...,...,...,...,...,...,...,...
2739,Terengganu,2022,property,theft_vehicle_lorry,20,nondomestic,173
2740,Terengganu,2022,property,theft_vehicle_motorcar,76,domestic,286
2741,Terengganu,2022,property,theft_vehicle_motorcar,76,nondomestic,173
2742,Terengganu,2022,property,theft_vehicle_motorcycle,444,domestic,286


In [18]:
# Check for null values in the merged dataset
print(merged_data.isnull().sum())

state                0
year                 0
category             0
type                 0
crimes               0
sector               0
water_consumption    0
dtype: int64


## 10 visualization ideas

1. Total Crimes per State Over the Years
- Graph Type: Line Chart
- Insight: Shows trends in crime rates for each state over time.

2. Water Consumption by Sector (Domestic vs. Non-Domestic) Over the Years
- Graph Type: Stacked Area Chart
- Insight: Helps compare how domestic and non-domestic water consumption has changed over time.

3. Crime Categories Breakdown
- Graph Type: Pie Chart or Bar Chart
- Insight: Highlights which crime category (e.g., assault, property crime) is the most common across all states.

4. Water Consumption vs. Crime Rate by State
- Graph Type: Scatter Plot
- Insight: Shows whether states with higher water consumption tend to have higher or lower crime rates.

5. Yearly Crime Trend by Type
- Graph Type: Line Chart
- Insight: Tracks trends for different crime types over the years.

6. Crime Rate vs. Water Consumption by Sector
- Graph Type: Box Plot or Heatmap
- Insight: Analyzes whether domestic or non-domestic water usage correlates more with crime rates.

7. Top 5 States with the Highest Crime Rates
- Graph Type: Horizontal Bar Chart
- Insight: Ranks states by the total number of crimes over all years.

8. Crime Type Distribution Across States
- Graph Type: Stacked Bar Chart
- Insight: Compares how different crime types are distributed in each state.

9. Correlation Between Water Consumption and Property Crimes
- Graph Type: Scatter Plot with Regression Line
- Insight: Tests if higher water consumption is linked to more property crimes.

10. Yearly Water Consumption and Crime Comparison
- Graph Type: Dual-Axis Line Chart
- Insight: Helps visualize if changes in water consumption over time align with changes in crime rates.

## Generate Latitude & Longitude for Each State

In [21]:
# Initialize the geocoder
geolocator = Nominatim(user_agent="geoapi")

# List of states from your dataset
states = ["Johor", "Kedah", "Kelantan", "Melaka", "Negeri Sembilan", "Pahang", 
          "Perak", "Perlis", "Pulau Pinang", "Sabah", "Sarawak", "Selangor", 
          "Terengganu", "Wilayah Persekutuan Kuala Lumpur"]

# Function to get latitude and longitude
def get_lat_lon(state):
    try:
        location = geolocator.geocode(f"{state}, Malaysia")  # Adjust for your country
        if location:
            return location.latitude, location.longitude
    except:
        return None, None
    return None, None

In [25]:
# Create a DataFrame
data = []
for state in states:
    lat, lon = get_lat_lon(state)
    data.append([state, lat, lon])
    sleep(1)  # To prevent API rate limits

df = pd.DataFrame(data, columns=["state", "latitude", "longitude"])

In [26]:
# Save to CSV
df.to_csv("state_coordinates.csv", index=False)

# Display the results
print(df)

                               state  latitude   longitude
0                              Johor  2.022882  103.311456
1                              Kedah  5.809826  100.671504
2                           Kelantan  5.402130  102.063597
3                             Melaka  2.329374  102.288096
4                    Negeri Sembilan  2.783190  102.192532
5                             Pahang  3.616882  102.599455
6                              Perak  4.812181  100.979791
7                             Perlis  6.486839  100.257762
8                       Pulau Pinang  5.406501  100.255908
9                              Sabah  5.425736  117.032639
10                           Sarawak  2.502385  112.954728
11                          Selangor  3.208330  101.304146
12                        Terengganu  4.863074  102.994930
13  Wilayah Persekutuan Kuala Lumpur  3.152659  101.702220


In [30]:
merged_data_coordinates = pd.read_csv("merged_water_crime.csv")
coordinates = pd.read_csv("state_coordinates.csv")

# Merge datasets on the "State" column
merged_data_coordinates = merged_data_coordinates.merge(coordinates, on="state", how="left")

# Save the updated dataset
merged_data_coordinates.to_csv("dataset_with_coordinates.csv", index=False)

In [31]:
# Display the first few rows of the updated dataset to verify the merge
merged_data_coordinates

,state,year,category,type,crimes,sector,water_consumption,latitude,longitude
0,Johor,2016,assault,all,5640,domestic,773,2.022882,103.311456
1,Johor,2016,assault,all,5640,nondomestic,513,2.022882,103.311456
2,Johor,2016,assault,causing_injury,1416,domestic,773,2.022882,103.311456
3,Johor,2016,assault,causing_injury,1416,nondomestic,513,2.022882,103.311456
4,Johor,2016,assault,murder,140,domestic,773,2.022882,103.311456
...,...,...,...,...,...,...,...,...,...
2739,Terengganu,2022,property,theft_vehicle_lorry,20,nondomestic,173,4.863074,102.994930
2740,Terengganu,2022,property,theft_vehicle_motorcar,76,domestic,286,4.863074,102.994930
2741,Terengganu,2022,property,theft_vehicle_motorcar,76,nondomestic,173,4.863074,102.994930
2742,Terengganu,2022,property,theft_vehicle_motorcycle,444,domestic,286,4.863074,102.994930


In [32]:
# Check for null values in the merged dataset
print(merged_data_coordinates.isnull().sum())

state                  0
year                   0
category               0
type                   0
crimes                 0
sector                 0
water_consumption      0
latitude             196
longitude            196
dtype: int64


In [34]:
# Check for rows where 'Latitude' or 'Longitude' is null
null_states = merged_data_coordinates[merged_data_coordinates['latitude'].isnull() | merged_data_coordinates['longitude'].isnull()]

# Print the states with null values
print(null_states[['state', 'latitude', 'longitude']])

        state  latitude  longitude
588  Malaysia       NaN        NaN
589  Malaysia       NaN        NaN
590  Malaysia       NaN        NaN
591  Malaysia       NaN        NaN
592  Malaysia       NaN        NaN
..        ...       ...        ...
779  Malaysia       NaN        NaN
780  Malaysia       NaN        NaN
781  Malaysia       NaN        NaN
782  Malaysia       NaN        NaN
783  Malaysia       NaN        NaN

[196 rows x 3 columns]


In [36]:
# Drop rows where the 'State' is 'Malaysia'
merged_data_coordinates_cleaned = merged_data_coordinates[merged_data_coordinates['state'] != 'Malaysia']

# Print the cleaned DataFrame (optional)
print(merged_data_coordinates_cleaned)

# Optionally save the cleaned data to a new CSV
merged_data_coordinates_cleaned.to_csv("cleaned_dataset.csv", index=False)

           state  year  category                      type  crimes   
0          Johor  2016   assault                       all    5640  \
1          Johor  2016   assault                       all    5640   
2          Johor  2016   assault            causing_injury    1416   
3          Johor  2016   assault            causing_injury    1416   
4          Johor  2016   assault                    murder     140   
...          ...   ...       ...                       ...     ...   
2739  Terengganu  2022  property       theft_vehicle_lorry      20   
2740  Terengganu  2022  property    theft_vehicle_motorcar      76   
2741  Terengganu  2022  property    theft_vehicle_motorcar      76   
2742  Terengganu  2022  property  theft_vehicle_motorcycle     444   
2743  Terengganu  2022  property  theft_vehicle_motorcycle     444   

           sector  water_consumption  latitude   longitude  
0        domestic                773  2.022882  103.311456  
1     nondomestic                513 

In [38]:
# Recheck for null values in the merged dataset
print(merged_data_coordinates_cleaned.isnull().sum())

state                0
year                 0
category             0
type                 0
crimes               0
sector               0
water_consumption    0
latitude             0
longitude            0
dtype: int64


# Eh voila! Let's plot our first map using geopspatial methods

In [7]:
import folium
from folium.plugins import HeatMap
import pandas as pd

In [8]:
# Load dataset
df = pd.read_csv("cleaned_dataset.csv")

# Define "Value" columns for each heatmap
df["Crime_Value"] = df["crimes"]
df["Water_Value"] = df["water_consumption"]
df["Hybrid_Value"] = df["crimes"] + (df["water_consumption"] / 100)  # Normalize water usage

In [9]:
# Function to create heatmap
def create_heatmap(df, value_column, title, filename):
    # Initialize map at the mean location
    map_center = [df['latitude'].mean(), df['longitude'].mean()]
    m = folium.Map(location=map_center, zoom_start=6)

    # Prepare heatmap data (lat, lon, value)
    heat_data = df[['latitude', 'longitude', value_column]].values.tolist()

    # Add heatmap layer
    HeatMap(heat_data, radius=15, blur=10, max_zoom=10).add_to(m)

    # Save and display map
    m.save(filename)
    print(f"{title} saved as {filename}")

## Generate each heatmap

In [10]:
# Generate each heatmap for crime
create_heatmap(df, "Crime_Value", "Crime Heatmap", "crime_heatmap.html")

Crime Heatmap saved as crime_heatmap.html


In [11]:
# Generate each heatmap for water consumption
create_heatmap(df, "Water_Value", "Water Consumption Heatmap", "water_heatmap.html")

Water Consumption Heatmap saved as water_heatmap.html


In [12]:
# Generate each heatmap for hybrid
create_heatmap(df, "Hybrid_Value", "Hybrid Crime & Water Heatmap", "hybrid_heatmap.html")

Hybrid Crime & Water Heatmap saved as hybrid_heatmap.html
